Benchmarking RMSF analysis speed using cloud & disk reading

First perform the steps in `benchmark_setup.ipynb`

Steps:

1. Open a `zarr.Group` object from the trajectory stored on disk
2. Open one from the trajectory uploaded to an AWS S3 bucket
3. Create an `mda.Universe` object for both zarr groups and one for the original .xtc trajectory
4. Perform the RMSF analysis for each `mda.Universe`, time, and compare results

In [ ]:
import zarrtraj
import MDAnalysis as mda
import zarr
from zarr.storage import LRUStoreCache
import s3fs
import os

# 1
yiipHDD = zarr.open_group("notebook_data_tmp/yiip.zarrtraj")

# 2
AWS_ACCESS_KEY_ID = os.getenv("AWS_ACCESS_KEY_ID")
AWS_SECRET_ACCESS_KEY = os.getenv("AWS_SECRET_ACCESS_KEY")
BUCKET_NAME = os.getenv("BUCKET_NAME")

s3 = s3fs.S3FileSystem(key=AWS_ACCESS_KEY_ID, secret=AWS_SECRET_ACCESS_KEY)
store = s3fs.S3Map(root='zarrtraj-test-data/s3-test-cacheing.zarrtraj', s3=s3, check=False)
# Select max_size value in bytes based on chunking of zarrtraj data
# At least one chunk must fit in the cache
cache = LRUStoreCache(store, max_size=2**25)
yiipS3 = zarr.open_group(store=cache)

In [ ]:
# 3
uHDD = mda.Universe("notebook_data_tmp/YiiP_system.pdb", yiipHDD)
uS3 = mda.Universe("notebook_data_tmp/YiiP_system.pdb", yiipS3)
uXTC = mda.Universe("notebook_data_tmp/YiiP_system.pdb", "YiiP_system_90ns_center.xtc")